In [3]:
import pandas as pd
import numpy as np

company_rank = "1"
company_name = "Walmart"
df = pd.read_csv(f"~/Desktop/workspace/indeed/Job-Satisfaction/data/companies/{company_rank}_{company_name}/"
                + f"{company_name}.csv")
df.head()
                

,Unnamed: 0,Index,Rating_Number,Work_Life,Benefits,Job_Advancement,Management,Culture,Review_Title,Reviewer_Job_Title,Reviewer_Job_Status,Reviewer_Job_Location,Reviewed_Date,Review_Text,Pros,Cons,Reviewed_Year
0,0,0,3.0,4.0,3.0,4.0,3.0,4.0,Decent,Service Manager,Former Employee,"Price, UT",2019-03-08,They are all different I got lucky and had a g...,NaN,NaN,2019
1,1,1,3.0,3.0,0.0,3.0,3.0,3.0,Walmart is great opportunity for people lookin...,Team Lead,Current Employee,"Charleston, SC",2019-07-13,Walmart is a good company with great benefits...,NaN,NaN,2019
2,2,2,4.0,4.0,4.0,4.0,3.0,4.0,It was a good place to work laid back had alot...,Stocker,Former Employee,"Riverdale, GA",2019-07-13,Just a good place to work i really enjoyed wor...,NaN,NaN,2019
3,3,3,5.0,0.0,0.0,0.0,0.0,0.0,Best job for anybody to work at,Walmart Stocker,Former Employee,"Portage, IN",2019-07-13,It's a fast paste job you have to keep up with...,NaN,NaN,2019
4,4,4,1.0,1.0,1.0,2.0,1.0,1.0,Not a place to work,Customer Service Representative,Former Employee,"Bellevue, WA",2019-07-13,Everything Walmart does is unethical. From as...,NaN,NaN,2019


In [4]:
df[['Reviewed_Date']].sort_values(by="Reviewed_Date").head(5)

,Reviewed_Date
156573,2011-02-25
156572,2011-03-08
156571,2011-03-14
156570,2011-04-01
156569,2011-04-06


In [5]:
df.shape

(156574, 17)

In [6]:
import re
import string

import nltk
from nltk import WordPunctTokenizer, PunktSentenceTokenizer, PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

sent_tokenizer = PunktSentenceTokenizer()
word_tokenizer = WordPunctTokenizer()
stemmer = LancasterStemmer()
lemmatizer = WordNetLemmatizer()

def data_clean(text):
    text = text.lower()
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text)
    text = re.sub('@[^\s]+', '', text)   
    text = text.replace("#", "")  
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'(.)\1+', r'\1\1', text)
    return text


def preprocess(reviews): 
    px_reviews = []
    reviews = [data_clean(review) for review in reviews] 
    for review in reviews: 
        review = word_tokenizer.tokenize(review)
        review = [term for term in review if term not in stopwords.words('english')]
        review = [stemmer.stem(term) for term in review]
        review = [lemmatizer.lemmatize(term, pos='v') for term in review]
        review = " ".join(review)
        px_reviews.append(review)
    
    return px_reviews

In [7]:
reviews = df['Review_Text'].sample(n=40000, random_state=42).tolist()
reviews[:5]

['I am able to learn and do what I needed to do. I went from customer service to DSD receiving and I enjoy the back room. I would come in clear my door for my vendors. Then I would clean my area so my workplace was set up the right way. When UPS or FedEx come I would scan them in sign for it. Then I would scan in the product give it to the department and do my paper work.',
 'The management was good to work for and all the co-workers and it was not hard at all and I love it.',
 'The labor performed at Walmart is minimal in order to keep associates at top-performance on the floor and in back-stock. Although the work performed within the stores is continuous and long-lasting, it is not extreme. This benefits the workers by maintaining good levels of energy and minimizing stress levels in order to perform with maximum efficiency and to give customers a positive shopping experience.',
 "Wal-Mart has a lot of benefits and gives advancements from my experience. I appreciated Wal-Mart as a ho

In [8]:
px_reviews = preprocess(reviews)

In [9]:
px_reviews[:5]

['abl learn nee go custom serv dsd receiv enjoy back room would com clear door vend would cle be workplac set right way up fedex com would scan sign would scan produc giv depart pap work',
 'man good work cowork hard lov',
 'lab perform walmart minim ord keep assocy topperform flo backstock although work perform within stor continu longlast extrem benefit work maintain good level energy minim stress level ord perform maxim efficy giv custom posit shop expery',
 'walmart lot benefit giv adv expery apprecy walmart hol hug turn away extrem poor man man would nee rar gav group speech morn mc40 wouldnt turn prop on would get pun act',
 'good day work good custom good learn work fast pac norm pac man gre fab man good work']

In [31]:
# from collections import Counter

# terms_list = [sent.split() for sent in px_reviews] 
# # terms[:5]
# terms = [item for sublist in terms_list for item in sublist]
# # terms[:5]

# counter_obj = Counter(terms)
# counter_obj.most_common(100)

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer(max_df=0.85, max_features=5000)
word_count_vector=cv.fit_transform(px_reviews)

In [33]:
# list(cv.vocabulary_.keys())[:1000]
d = cv.vocabulary_

In [36]:
px_reviews = [review for review in px_reviews if len(review.split()) > 0]

In [37]:
len(px_reviews)

40000

In [35]:
px_reviews = [review.split() for review in px_reviews]
px_reviews[:5]

[['abl',
  'learn',
  'nee',
  'go',
  'custom',
  'serv',
  'dsd',
  'receiv',
  'enjoy',
  'back',
  'room',
  'would',
  'com',
  'clear',
  'door',
  'vend',
  'would',
  'cle',
  'be',
  'workplac',
  'set',
  'right',
  'way',
  'up',
  'fedex',
  'com',
  'would',
  'scan',
  'sign',
  'would',
  'scan',
  'produc',
  'giv',
  'depart',
  'pap',
  'work'],
 ['man', 'good', 'work', 'cowork', 'hard', 'lov'],
 ['lab',
  'perform',
  'walmart',
  'minim',
  'ord',
  'keep',
  'assocy',
  'topperform',
  'flo',
  'backstock',
  'although',
  'work',
  'perform',
  'within',
  'stor',
  'continu',
  'longlast',
  'extrem',
  'benefit',
  'work',
  'maintain',
  'good',
  'level',
  'energy',
  'minim',
  'stress',
  'level',
  'ord',
  'perform',
  'maxim',
  'efficy',
  'giv',
  'custom',
  'posit',
  'shop',
  'expery'],
 ['walmart',
  'lot',
  'benefit',
  'giv',
  'adv',
  'expery',
  'apprecy',
  'walmart',
  'hol',
  'hug',
  'turn',
  'away',
  'extrem',
  'poor',
  'man',
  'm

In [36]:
data_px = px_reviews.copy()

# LDA

In [47]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
from pprint import pprint

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import os
root = os.path.expanduser('~')

In [44]:
# Create Dictionary
id2word = corpora.Dictionary(data_px)

# Create Corpus
texts = data_px

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:5])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 4)], [(28, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1)], [(6, 1), (12, 1), (28, 2), (32, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 2), (49, 1), (50, 1), (51, 1), (52, 2), (53, 2), (54, 3), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1)], [(12, 1), (15, 1), (30, 2), (35, 2), (39, 1), (43, 1), (44, 1), (60, 2), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 2), (81, 1)], [(6, 1), (14, 1), (28, 3), (32, 4), (35, 2), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 2)]]


In [45]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:4]]

[[('abl', 1),
  ('back', 1),
  ('be', 1),
  ('cle', 1),
  ('clear', 1),
  ('com', 2),
  ('custom', 1),
  ('depart', 1),
  ('door', 1),
  ('dsd', 1),
  ('enjoy', 1),
  ('fedex', 1),
  ('giv', 1),
  ('go', 1),
  ('learn', 1),
  ('nee', 1),
  ('pap', 1),
  ('produc', 1),
  ('receiv', 1),
  ('right', 1),
  ('room', 1),
  ('scan', 2),
  ('serv', 1),
  ('set', 1),
  ('sign', 1),
  ('up', 1),
  ('vend', 1),
  ('way', 1),
  ('work', 1),
  ('workplac', 1),
  ('would', 4)],
 [('work', 1),
  ('cowork', 1),
  ('good', 1),
  ('hard', 1),
  ('lov', 1),
  ('man', 1)],
 [('custom', 1),
  ('giv', 1),
  ('work', 2),
  ('good', 1),
  ('although', 1),
  ('assocy', 1),
  ('backstock', 1),
  ('benefit', 1),
  ('continu', 1),
  ('efficy', 1),
  ('energy', 1),
  ('expery', 1),
  ('extrem', 1),
  ('flo', 1),
  ('keep', 1),
  ('lab', 1),
  ('level', 2),
  ('longlast', 1),
  ('maintain', 1),
  ('maxim', 1),
  ('minim', 2),
  ('ord', 2),
  ('perform', 3),
  ('posit', 1),
  ('shop', 1),
  ('stor', 1),
  ('stress',

In [53]:
num_topic = 12
# num_topic = 10
# %%time
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = root + '/Downloads/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topic, id2word=id2word,  
                                            random_seed=42)

In [54]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamallet, texts=data_px, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5977096715696119


In [55]:
##mallet model to lda model conversion

optimal_model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)

In [56]:
# %%time
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)
vis

/Users/bishalsainju/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/Users/bishalsainju/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/Users/bishalsainju/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
/Users/bishalsainju/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
2     -0.197501  0.219380       1        1  8.797463
9     -0.184206 -0.195467       2        1  8.679391
5     -0.072044  0.129081       3        1  8.633724
7     -0.203875  0.230444       4        1  8.628919
8     -0.011924  0.102013       5        1  8.553440
0     -0.125800 -0.069986       6        1  8.407496
4     -0.143071 -0.287353       7        1  8.298634
3     -0.014251 -0.193171       8        1  8.261135
10     0.113035  0.034836       9        1  8.122756
11     0.255782  0.032368      10        1  7.922700
6      0.314341  0.011409      11        1  7.847207
1      0.269515 -0.013552      12        1  7.847134, topic_info=    Category          Freq    Term         Total  loglift  logprob
28   Default  56835.000000    work  56835.000000  30.0000  30.0000
6    Default  19024.000000  custom  19024.000000  29.0000  29.0000
35   Default  27245.000000     man  27245.000000  28.0000  28.0000
176  Default  24035.000000     job  24035.000000  27.0000  27.0000
32   Default  12013.000000    good  12013.000000  26.0000  26.0000
..       ...           ...     ...           ...      ...      ...
316  Topic12    387.679473   excel    885.482235   1.7191  -5.3047
138  Topic12    831.395213     wel   3972.542245   0.9810  -4.5418
12   Topic12    707.886708     giv   3137.980329   1.0560  -4.7026
584  Topic12    471.162074     lif   1612.305572   1.3148  -5.1097
600  Topic12    425.418183  provid   1475.381842   1.3014  -5.2118

[989 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
1225       5  1.000813      1
88         5  0.995778     10
88         8  0.003452     10
8897       9  1.077866   1020
4265       2  1.008599    110
...      ...       ...    ...
1060       9  0.999544  young
10625     12  0.874434    yum
386        1  0.999316    zon
331       11  1.000145      ’
819        4  0.999311      •

[1261 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 10, 6, 8, 9, 1, 5, 4, 11, 12, 7, 2])